In [1]:
from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import Draw

## Numerics

### Exercise 8.

We want to solve the PDE-constrained shape optimization problem

$$
\min_{\Omega \subset \mathsf{D}} J(u)
:= \int_\Omega |u_\Omega - u_{ref}|^2 \, dx
$$

subject to $(\Omega,u_\Omega)$ satisfying

$$
\int_\Omega \nabla u_\Omega \cdot \nabla v + u_\Omega v \, dx
=
\int_\Omega f v \, dx
\quad \text{for all } v \in H_0^1(\Omega),
$$

where $\Omega \subset \mathbb{R}^2$ and
$u_d, f \in C^1(\mathbb{R}^2)$.


In [2]:
geo = unit_square.GenerateMesh(maxh=0.1)
mesh = Mesh(geo)
#Draw(mesh)

f = CoefficientFunction(1)
ud = exp(-4**2*((y-0.5)**2+ (x-0.5)**2))

grad_f = CoefficientFunction( (f.Diff(x), f.Diff(y) ) )
grad_ud = CoefficientFunction( (ud.Diff(x), ud.Diff(y) ) )

In [3]:
fes = H1(mesh, order=2, dirichlet=".*")
u, v = fes.TnT()
gfu = GridFunction(fes)
scene = Draw (gfu, mesh, "state")

a = BilinearForm(fes, symmetric=True)
a += grad(u)*grad(v)*dx
a += u*v*dx

fstate = LinearForm(fes)
fstate += f*v*dx

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

In [4]:
def SolveStateEquation():
    rhs = gfu.vec.CreateVector()
    rhs.data = fstate.vec - a.mat * gfu.vec
    update = gfu.vec.CreateVector()
    update.data = a.mat.Inverse(fes.FreeDofs()) * rhs
    gfu.vec.data += update

In [5]:
a.Assemble()
fstate.Assemble()
SolveStateEquation()
scene.Redraw()

### Adjoint equation

The adjoint p satisfies

$$
\int_\Omega \nabla w \cdot \nabla p + w p \, dx
=
-2 \int_\Omega (u_\Omega - u_{ref}) w 
\quad \text{for all } w \in H_0^1(\Omega),
$$

In [6]:
def Cost(u):
    return (u - ud)**2 * dx

p, w = fes.TnT()
gfp = GridFunction(fes)
scene = Draw (gfp, mesh, "adjoint")

fadjoint = LinearForm(fes)
#fadjoint += -1*Cost(gfu).Diff(gfu,w)
fadjoint += -2*(gfu - ud)*w*dx

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

In [7]:
def SolveAdjointEquation():
    # can use A since diff op is self adjoint
    rhs = gfp.vec.CreateVector()
    rhs.data = fadjoint.vec - a.mat.T * gfp.vec
    update = gfp.vec.CreateVector()
    update.data = a.mat.Inverse(fes.FreeDofs()).T * rhs
    gfp.vec.data += update

In [8]:
fadjoint.Assemble()
SolveAdjointEquation()
scene.Redraw()

The shape derivative of the shape function

$$
\mathcal{J}(\Omega) := \int_\Omega |u - u_d|^2 \, dx
$$

at $\Omega$ in direction $X$ is given by

$$
\begin{aligned}
d\mathcal{J}(\Omega; X)
=& \int_{\Omega} \operatorname{div}(X)\, |u - u_d|^2
- 2 (u - u_d)\, \nabla u_d \cdot X \, dx \\
&+ \int_{\Omega}
\bigl( \operatorname{div}(X) I - D X - D X^\top \bigr)
\nabla u \cdot \nabla p \, dx \\
&+ \int_\Omega \operatorname{div}(X) u p \, dx \\
&- \int_{\Omega}
\bigl( \nabla f \cdot X + f \operatorname{div}(X) \bigr) p \, dx .
\end{aligned}
$$

We now assemble this shape derivative in NGSolve as follows:


In [9]:
VEC = H1(mesh, order=2, dim=2)
PHI, X = VEC.TnT()

dJOmega = LinearForm(VEC)
dJOmega += SymbolicLFI(div(X)*((gfu - ud)**2) - 2*grad_ud*X*(gfu - ud))
dJOmega += SymbolicLFI((div(X)*InnerProduct(grad(gfu),grad(gfp))))
dJOmega += SymbolicLFI((-InnerProduct(X.Deriv()*grad(gfu), grad(gfp))))
dJOmega += SymbolicLFI((-InnerProduct(grad(gfu),X.Deriv()*grad(gfp))))      # Transpose by shifting in scalar product
dJOmega += SymbolicLFI(div(X)*gfu*gfp)
dJOmega += SymbolicLFI(-InnerProduct(grad_f,X)*gfp)
dJOmega += SymbolicLFI(-f*div(X)*gfp)

- Next, we want to find a vector field $X$ which yields a decrease of the objective
  function, i.e. we want to find a vector field $X$ such that

  $$
  d\mathcal{J}(\Omega; X) < 0.
  $$

- This can be achieved by solving an auxiliary boundary value problem of the type

  $$
  \text{Find } X \in H :
  \qquad
  b(X, \Phi) = d\mathcal{J}(\Omega; \Phi)
  \quad \text{for all } \Phi \in H,
  $$

  for a suitable Hilbert space $H$ (e.g. $H = H^1(\Omega)^2$).
  Here, $b(\cdot,\cdot) : H \times H \to \mathbb{R}$ is a positive definite bilinear
  form which we are free to choose. Then $-X$ is a descent direction since

  $$
  d\mathcal{J}(\Omega; -X)
  = - d\mathcal{J}(\Omega; X)
  = - b(X, X)
  < 0.
  $$

- For now, we start with the bilinear form

  $$
  b(X, Y)
  =
  \int_\Omega \nabla X : \nabla Y + X \cdot Y \, dx .
  $$


In [10]:
def pose_deformation_equation(gamma=None): 
    # Bilinearform of Exercise 8/9
    b = BilinearForm(VEC)
    b += InnerProduct(grad(X),grad(PHI))*dx + InnerProduct(X,PHI)*dx
    if gamma:
        print('Using gamma =', gamma)
        b += gamma * InnerProduct(CF((-X.Diff(x)+X.Diff(y), X.Diff(x)+X.Diff(y))), CF((-PHI.Diff(x)+PHI.Diff(y), PHI.Diff(x)+PHI.Diff(y)))) * dx
    #print("Expression tree \n", b)
    return b

def SolveDeformationEquation():
    rhs = gfX.vec.CreateVector()
    rhs.data = dJOmega.vec - b.mat * gfX.vec
    update = gfX.vec.CreateVector()
    update.data = b.mat.Inverse(VEC.FreeDofs()) * rhs
    gfX.vec.data += update

In [11]:
def optimize_shape():
    gfset.Set((0,0))
    mesh.SetDeformation(gfset)
    scene.Redraw()
    a.Assemble()
    fstate.Assemble()
    SolveStateEquation()

    LineSearch = True

    iter_max = 600
    Jold = Integrate(Cost(gfu), mesh)
    converged = False

    # input("Press enter to start optimization")
    print('Cost at initial design', Integrate (Cost(gfu), mesh))
    for k in range(iter_max):
        mesh.SetDeformation(gfset)
        scene.Redraw()

        print('cost at iteration', k, ': ', Jold)

        a.Assemble()
        fstate.Assemble()
        SolveStateEquation()

        fadjoint.Assemble()
        SolveAdjointEquation()

        b.Assemble()
        dJOmega.Assemble()
        SolveDeformationEquation()

        scale = 0.01 / Norm(gfX.vec)
        gfsetOld = gfset
        gfset.vec.data -= scale * gfX.vec

        Jnew = Integrate(Cost(gfu), mesh)

        if LineSearch:
            while Jnew > Jold and scale > 1e-12:
                #input('a')
                scale = scale / 2
                print("scale = ", scale)
                #print("scale without normalization factor = ", scale* Norm(gfX.vec))
                if scale <= 1e-12:
                    converged = True
                    break

                gfset.vec.data = gfsetOld.vec - scale * gfX.vec         # scale times -X is descent direction
                mesh.SetDeformation(gfset)                              # applying deformation and solve new state

                a.Assemble()                                           # note that during line search we only have to solve state to check if update yields lower costs              
                fstate.Assemble()
                SolveStateEquation()
                Jnew = Integrate(Cost(gfu), mesh)
                print("New cost during line search: ", Jnew)

        if converged==True:
            print("No more descent can be found")
            break
        Jold = Jnew

        Redraw(blocking=True)

    print("Norm of gset", Integrate(gfset**2, mesh))

In [12]:
b = pose_deformation_equation(gamma= 3)
gfX = GridFunction(VEC)

# gfset denotes the deformation of the original domain and will be updated during the shape optimization
gfset = GridFunction(VEC)
gfset.Set((0,0))
scene = Draw(gfset,mesh,"gfset")
SetVisualization (deformation=True)

optimize_shape()

Using gamma = 3


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

Cost at initial design 0.07797736156507301
cost at iteration 0 :  0.07797736156507301
cost at iteration 1 :  0.07796153130230203
cost at iteration 2 :  0.07786485030893471
cost at iteration 3 :  0.07776817189590422
cost at iteration 4 :  0.07767149889477505
cost at iteration 5 :  0.07757483411583313
cost at iteration 6 :  0.0774781803488548
cost at iteration 7 :  0.0773815403638579
cost at iteration 8 :  0.07728491691183546
cost at iteration 9 :  0.07718831272547264
cost at iteration 10 :  0.07709173051984639
cost at iteration 11 :  0.07699517299310918
cost at iteration 12 :  0.07689864282715651
cost at iteration 13 :  0.07680214268827837
cost at iteration 14 :  0.07670567522779552
cost at iteration 7 :  0.0773815403638579
cost at iteration 8 :  0.07728491691183546
cost at iteration 9 :  0.07718831272547264
cost at iteration 10 :  0.07709173051984639
cost at iteration 11 :  0.07699517299310918
cost at iteration 12 :  0.07689864282715651
cost at iteration 13 :  0.07680214268827837
cost 